# reference
[example](https://github.com/peterdsharpe/AeroSandbox/blob/master/tutorial/06%20-%20Aerodynamics/01%20-%20AeroSandbox%203D%20Aerodynamics%20Tools/01%20-%20Vortex%20Lattice%20Method.ipynb)

In [5]:
import aerosandbox as asb
import aerosandbox.numpy as np
%matplotlib notebook

In [ ]:
# Vad vill vi egentligen göra?
1. Hitta en flygplansprofil som ger ca 800g lyftkraft i 15-20km/h
1. minimera turbolens/drag för att få så lång flygtid som möjligt

In [6]:
wing_airfoil = asb.Airfoil("sd7037")

In [11]:
cord1 = 0.5
l1 = 0.4
x1 = 0.3
cord2 = 0.2
l2 = 0.7
x2 = 0.5
cord3 = 0.15

airplane = asb.Airplane(
    name="OdDa-wing",
    xyz_ref=[0, 0, 0],  # CG location
    wings=[asb.Wing(
            name="Main Wing",
            symmetric=True,  # Should this wing be mirrored across the XZ plane?
            xsecs=[  # The wing's cross ("X") sections
                asb.WingXSec(  # Root
                    xyz_le=[0, 0, 0],  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    chord=cord1,
                    twist=2,  # degrees
                    airfoil=wing_airfoil,  # Airfoils are blended between a given XSec and the next one.
                ),
                asb.WingXSec(  # Mid
                    xyz_le=[x1, l1, 0],
                    chord=cord2,
                    twist=0,
                    airfoil=wing_airfoil,
                ),
                asb.WingXSec(  # Tip
                    xyz_le=[x2, l2, 0.0],
                    chord=cord3,
                    twist=-2,
                    airfoil=wing_airfoil,
                ),
                ],
            )
        ]
)


In [12]:
vlm = asb.VortexLatticeMethod(
    airplane=airplane,
    op_point=asb.OperatingPoint(
        velocity=25,  # m/s
        alpha=5,  # degree
    )
)



In [13]:
aero = vlm.run()  # Returns a dictionary
for k, v in aero.items():
    print(f"{k.rjust(4)} : {v}")


   L : 89.94097178956552
   D : 3.459046195289738
   Y : -1.314188558323992e-16
   l : 2.435501049717537e-15
   m : -29.464162221016284
   n : 4.482090722847888e-16
  CL : 0.6102265862153956
  CD : 0.023468747438616107
  CY : -8.916434653004498e-19
  Cl : 1.1802589582613334e-17
  Cm : -0.628274808901644
  Cn : 2.1720490442797218e-18
 F_g : [-4.39298872e+00 -1.31418856e-16  8.99001950e+01]
 F_w : (-3.459046195289738, -1.314188558323992e-16, -89.94097178956552)
 M_g : [-2.38716924e-15 -2.94641622e+01 -6.58771404e-16]
 M_w : (2.435501049717537e-15, -29.464162221016284, 4.482090722847888e-16)


In [14]:
vlm.draw(show_kwargs=dict(jupyter_backend='panel'))

In [1]:
from pyvista import demos

# basic glyphs demo
mesh = demos.glyphs(2)

text = demos.logo.text_3d("I'm interactive!", depth=0.2)
text.points *= 0.1
text.translate([0, 1.4, 1.5], inplace=True)
mesh += text
mesh['Example Scalars'] = mesh.points[:, 0]

mesh.plot(cpos='xy', jupyter_backend='ipygany', show_scalar_bar=True)


AppLayout(children=(VBox(children=(HTML(value='<h3>Example Scalars</h3>'), Dropdown(description='Colormap:', o…

In [4]:
from pyvista import demos
demos.plot_logo(jupyter_backend='panel')